# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

How to sanity check the environment?


In [7]:
var deps = ['npm install', 'JSONStream', 'path', 'fs', 'vm', ];
require('child_process').execSync(deps.slice(0, 2).join(' '));
deps.slice(1).forEach(v => (global[v] = require(v)));

// TODO: exports, require, module, __filename, __dirname
$$.done('import ready');


How to import a code cells from jupyter?

How to import cells of different {types}?


In [8]:
var getCells = (notebook, types) => {
    if (typeof types == 'undefined') {
        var types = 'code';
    }
    if (typeof types === 'string') {
        types = [types];
    }
    var cellsToRun = [];
    var cells = fs.createReadStream(notebook)
        .pipe(require('JSONStream').parse('cells.*'));
    cells.on('data', (cell) => {
        if(types.indexOf(cell.cell_type) > -1) {
            cellsToRun[cellsToRun.length] = cell;
        }
    });
    return new Promise((resolve, reject) => {
        cells.on('error', e => reject(e, notebook));
        cells.on('close', () => resolve(cellsToRun, notebook))
    });
};
$$.done(getCells);



How to create VM promises out of the cell content?

In [9]:
// supported format How to find answers.ipynb[2]
var promiseContext = (scripts, notebook) => {
    var tempGlobal = Object.assign({}, global);
    tempGlobal.$$ = Object.assign({}, $$);
    var context = vm.createContext(tempGlobal);
    return scripts.map((cell, i) => {
        var script = new vm.Script(`\n\n;\n\n` + cell + '\n\n;\n\n');
        return ((resolve, reject) => {
            var result;
            try {
                var tmp$$ = Object.assign(tempGlobal.$$, {
                    _isAsync: false,
                    async: () => tmp$$._isAsync = true,
                    done: (d) => resolve(d, i)
                });
                tempGlobal.$$ = tmp$$;
                result = script.runInContext(context);
                if (!tmp$$._isAsync) {
                    return resolve(result, i);
                }
            } catch (e) {
                return resolve(e, i);
            }
        });
    });
};

$$.async();
var cell = [
    'console.log("Sucessfully ran a script in it\'s own context!")'
];
var promise = promiseContext(cell, 'this one')
    .map((p) => new Promise(p))[0]
    .then(() => $$.done(promiseContext));


How to resolve a list of promises?

How to execute all the cells in a notebook?



In [10]:

var runAllPromises = (promises, notebook) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (new Promise(func)).then((r, i) => {
                return r
            }).then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([])).catch(e => console.log(e));
};

$$.async();
var cells = [
    '$$.done("promise 1")',
    '$$.done("promise 2")',
    '$$.done("Sucessfully ran a all promises!")'
];
var promises = promiseContext(cells, 'this one')
runAllPromises(promises, 'this one')
    .then((results) => {
        $$.done(runAllPromises);
        return results;
    });


How to import a {notebook}?

How to test the import feature?


In [14]:
if (typeof importedNotebooks == 'undefined') {
    var importedNotebooks = [];
}
if (typeof importedFunctions == 'undefined') {
    var importedFunctions = {};
}
var importNotebook = (notebook) => {
    if(typeof notebook === 'undefined') {
        return Promise.resolve();
    }
    if (importedNotebooks.indexOf(notebook) > -1) {
        console.log('Already imported ' + importedNotebooks.toString());
        return Promise.resolve(importedFunctions[notebook]);
    }
    return getCells(notebook, 'code').then((cells) => {
        var sources = cells.map(cell => cell.source.join('').trim());
        var promises = promiseContext(sources, notebook);
        console.log('started notebook ' + notebook + ' with ' + cells.length + ' cells');
        importedNotebooks[importedNotebooks.length] = notebook;
        return runAllPromises(promises, notebook);
    }).then((results) => {
        var result = {};
        results.forEach((r, i) => {
            if (typeof r === 'function') {
                result[notebook+'['+i+']'] = r;
                global[notebook+'['+i+']'] = r;
            }
        });
        importedFunctions[notebook] = result;
        return result;
    }).catch(e => console.log(e));
}

try {
    // test to make sure it works
    $$.async();
    var imports = importNotebook('How to find answers.ipynb')
            .then((results) => {
                console.log(results);
                $$.done(importNotebook);
                return results;
            }).catch((e) => $$.done(e));
} catch (e) {$$.done(e)}


These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?



In [12]:
//$$.async();
//importNotebook('How to interpret questions.ipynb')
//    .then(() => $$.done('done importing importing'));

